## Pore size and flow rate distributions in 2D porous media calculator

In [ ]:
import os
import numpy as np
import pyvista as pv
from scipy.ndimage import distance_transform_edt
from scipy.spatial import KDTree
from skimage.morphology import medial_axis
from shapely import vectorized
from shapely.geometry import Point, LineString, box
from shapely.ops import polygonize, unary_union
from shapely.prepared import prep
import networkx as netx
import matplotlib.pyplot as plt

In [ ]:
# Leemos y cargamos el VTK

case_dir = os.path.expanduser("~/OpenFOAM/jose-v2406/run/YDRAY-flow_n13_sat")
vtk_file = os.path.join(case_dir, "VTK", "YDRAY-flow_n13_sat_1047.vtm")

mb        = pv.read(vtk_file)
vol_mesh  = mb["internal"]
cyl_patch = mb["boundary"]["wallFluidSolid"]

_, _, _, _, zmin, zmax = vol_mesh.bounds
z_mid = 0.5 * (zmin + zmax)

obst_section = (
    cyl_patch.extract_surface()
             .slice(normal="z", origin=(0, 0, z_mid))
             .clean()
)
dom_section = (
    vol_mesh.extract_surface()
            .slice(normal="z", origin=(0, 0, z_mid))
            .clean()
)

print(f"Obstáculos en corte: {obst_section.n_lines:,d} aristas")

In [ ]:
# VISUALIZACIÓN

WINDOW_SIZE = (3200, 2400)   # resolución ventana
AXIS_FONT   = 40              # tamaño de números

pl = pv.Plotter(window_size=WINDOW_SIZE)
pl.set_background("white")

pl.add_mesh(obst_section, color="blue",  line_width=1.5, label="Obstacles")
pl.add_mesh(dom_section,  color="gray", line_width=0.8, opacity=0.4,
            label="Domain boundary")



axes = pl.show_bounds(grid="front", fmt="%.2f", font_size=AXIS_FONT)


axes.x_label_offset = 400    # mueve los números del eje X hacia abajo
axes.y_label_offset = 400    # mueve los números del eje Y hacia la izquierda

pl.view_xy()

pl.show(window_size=WINDOW_SIZE)

In [ ]:
# Poligonizamos con shapely el boundary y los obstaculos

def polydata_to_polygon(polydata):
    # toma un polydata y devuelve los poligonos resultantes
    if isinstance(polydata, pv.UnstructuredGrid):
        polydata = polydata.extract_geometry()
    total_entries = len(polydata.lines)
    print(f"Entradas en polydata.lines: {total_entries}")
    lines = []
    i, count = 0, 0
    while i < total_entries:
        n_pts = polydata.lines[i]
        idx = polydata.lines[i+1:i+1+n_pts]
        coords = [tuple(polydata.points[j][:2]) for j in idx]
        lines.append(LineString(coords))
        i += 1 + n_pts
        count += 1
        if count % 500 == 0:
            print(f"Procesadas {count} lineas, i = {i}/{total_entries}")
    merged = unary_union(lines)
    polys = list(polygonize(merged))
    return polys


# Nos quedamos solo con los contornos de los obstaculos circulares
clean_obs = obst_section.clean(tolerance=1e-6)
conn = clean_obs.connectivity() 
n_beads = int(conn['RegionId'].max())+1
print(f"Obtenidos {n_beads} contornos de obstaculo")



In [ ]:
# Poligonalizamos cada bead por separado
obs_polys = []
for rid in range(n_beads):
    bead = conn.threshold([rid, rid], scalars='RegionId').clean(tolerance = 1e-6).extract_geometry()
    polys = polydata_to_polygon(bead)
    if polys:
        obs_polys.extend(polys)

print(f"En total, {len(obs_polys)} obstaculos convertidos a poligono")

In [ ]:

xmin, xmax, ymin, ymax, zmin, zmax = dom_section.bounds
x_cutoff = max(poly.bounds[2] for poly in obs_polys)
xmax = x_cutoff
print(f"La esquina inf. izda. esta en ({xmin}, {ymin}), la esquina sup. dcha. esta en ({xmax}, {ymax})")




# Unimos los obstaculos y definimos la zona por la que fluye el agua
outer = box(xmin, ymin, xmax, ymax)
beads = unary_union(obs_polys)
free_region = outer.difference(beads)


In [ ]:
# Visualizamos los poligonos creados para asegurarnos de que esta todo correcto

fig, ax = plt.subplots(figsize=(12,8))

# Visualizacion del contorno exterior:
if outer.geom_type == 'Polygon':
    x, y = outer.exterior.xy
    ax.plot(x, y, color='blue', linewidth=2)
else:
    for poly in outer.geoms:
        x, y = poly.exterior.xy
        ax.plot(x, y, color='blue', linewidth=2)

#  Visualizacion de los beads:
if beads.geom_type == 'Polygon':
    xb, yb = beads.exterior.xy
    ax.plot(xb, yb, color='blue', linewidth=1)
else:
    for poly in beads.geoms:
        xb, yb = poly.exterior.xy
        ax.plot(xb, yb, color='blue', linewidth=1)
        


        

In [ ]:
# AHORA: CREAMOS UNA MASCARA BOOLEANA PARA SABER SI LOS PUNTOS ESTAN DENTRO O FUERA DEL ESPACIO LIBRE

# Queremos maxima resolucion: estimamos primero la longitud de celda mas pequena de nuestro mesh (dx)
# Para ello, construimos un KDTree y pedimos la lista de distancias al 2o vecino (el 1o es el propio punto)
pts2d = dom_section.points[:, :2]
tree = KDTree(pts2d)
dists, _ = tree.query(pts2d, k=2)
nn_dists = dists[:, 1]
# En vez de el valor minimo, tomamos el percentil 1 para evitar outliers:
dx = np.percentile(nn_dists, 1)
dy = dx # asumimos malla isotropica

print(f"La anchura de nuestro pixel sera de dx = {dx:.5g} m")

# Ahora si: definimos la resolucion de acuerdo a dx. Esta resolucion es cuasi maxima
nx = int(np.ceil((xmax-xmin)/dx)) + 1
ny = int(np.ceil((ymax-ymin)/dy)) + 1

print(f"Resolucion de la mascara: {nx}x{ny} (lo mas cercano al mallado original. Cristo murio por nosotros.)")

# Y AHORA: GENERAMOS LA MASCARA BOOLEANA
xs = np.linspace(xmin, xmax, nx)
ys = np.linspace(ymin, ymax, ny)
XX, YY = np.meshgrid(xs, ys, indexing="xy")

prepped = prep(free_region)

from shapely import contains_xy
mask = contains_xy(free_region, XX, YY)


print("Mascara creada")



In [ ]:
# Visualizamos la mascara creada para asegurarnos de que esta todo en orden

fig, ax = plt.subplots(figsize=(120,80))
im = ax.imshow(mask, origin='lower', extent=(xmin, xmax, ymin, ymax), cmap='gray', interpolation='nearest')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('White = free space')
plt.tight_layout()
plt.show()

In [ ]:
#AHORA: COMPUTAMOS EL ESQUELETO DEL MEDIO POROSO

# Primero, calculamos la distancia euclidea del centro cada pixel blanco (free) al centro del pixel negro 
# (obstacle) mas cercano. La funcion que usamos calcula estas distancias en "anchos de pixel" (dx)
dist_pixels = distance_transform_edt(mask)

# Ahora, computamos el esqueleto y proyectamos la "matriz de distancias" sobre el esqueleto
skeleton, skel_dist_pixels = medial_axis(mask, return_distance=True)

# Lo convertimos todo a metros
dist_map = dist_pixels * dx
skel_dist = skel_dist_pixels * dx

# Cuantos pixeles tiene el esqueleto?
n_skel_pts = skeleton.sum()
print(f"Puntos en el esqueleto: {n_skel_pts:,}")



In [ ]:
# Histograma rapido (pochillo) de half-widths
half_widths_mm = skel_dist[skeleton]*1e3

# Ajustamos a una Rayleigh
from scipy.stats import rayleigh, norm
loc_hat, sigma_hat = rayleigh.fit(half_widths_mm)

# Ajustamos a una Gaussiana
mu_g, sigma_g = norm.fit(half_widths_mm)

# Ajuste gaussiano con media y desviacion muestrales
mean_hw = half_widths_mm.mean()
std_hw = half_widths_mm.std()

# A pintar!
fig, ax = plt.subplots()
ax.hist(half_widths_mm, bins=50, density=True, alpha=0.6, label="Simulation")
r = np.linspace(0, half_widths_mm.max(), 200)
ax.plot(r, rayleigh.pdf(r, loc=loc_hat, scale=sigma_hat), 'r-', lw=2, label=f"Rayleigh fit\nσ={sigma_hat:.2f} mm")
ax.plot(r, norm.pdf(r, loc=mu_g, scale=sigma_g), 'b--', lw=2, label=f"Normal fit\nμ={mu_g:.2f} mm\nσ={sigma_g:.2f} mm")
ax.plot(r, norm.pdf(r, loc=mean_hw, scale=std_hw), 'g:', lw=2, label=f"Empiric normal\nμ={mean_hw:.2f} mm\nσ={std_hw:.2f} mm")
ax.set_xlabel("Half-width (mm)")
ax.set_ylabel("Probability density")
ax.legend()
plt.show()

print("Número de halfwidths:", len(half_widths_mm))
# El numero es mucho mayor que la cantidad total de Poiseuille tubes porque estamos considerando la 
# distancia de cada punto del skeleton al punto de obstaculo mas cercano como una half-width. 
# Sin embargo, se puede argumentar que esto es mas realista que tomar solo una distancia por tubo

In [ ]:
# Visualizamos el esqueleto sobre la mascara booleana para ver que todo esta correcto
plt.figure(figsize=(12,8), dpi=1200)
plt.imshow(mask, origin='lower', cmap='gray', extent=(xmin, xmax, ymin, ymax))
# el esqueleto:
ys, xs = np.where(skeleton)
plt.scatter(xs*dx + xmin, ys*dx + ymin, s=0.005, c='red', label='Skeleton')

In [ ]:

import numpy as np
from scipy import ndimage as ndi

def prune_spurs(skel, n_iters):
    """
    Elimina iterativamente los endpoints (píxeles de grado 1)
    del skeleton binario `skel`, `n_iters` veces.
    """
    # Un structuring element 3×3 de unos (contaremos vecinos)
    se = np.ones((3,3), dtype=int)
    sk = skel.copy()
    for _ in range(n_iters):
        # Convolution: cuenta vecinos (incluido el píxel)
        count = ndi.convolve(sk.astype(int), se, mode='constant', cval=0)
        # Los endpoints son píxeles en sk con exactamente 2 en count
        # porque count = 1(píxel) + 1(vecino) = 2
        endpoints = (sk & (count == 2))
        if not endpoints.any():
            break
        sk[endpoints] = False
    return sk

# Uso:
skeleton_pruned = prune_spurs(skeleton, 200)
skeleton = skeleton_pruned

# CONVERTIMOS EL SKELETON EN UN GRAFO
# en el que los skeleton pixels son los nodos y existe un link entre dos de ellos si son vecinos en la mascara 2D

import networkx as netx


orthogonal = [(0, 1), (0, -1), (1, 0), (-1, 0)]
diagonals  = [(1, 1), (1, -1), (-1, 1), (-1, -1)]

G = netx.Graph()

# extraemos el numero de fila (i) y el numero de columna (j) de cada pixel del skeleton:
coords = np.argwhere(skeleton)


# 1) Añade todos los nodos
for j, i in coords:
    G.add_node((i, j), width=skel_dist[j, i])

# 2) Construye las aristas con regla de diagonales condicionadas
for j, i in coords:
    this = (i, j)
    neighbors = set()
    # 2a) Vecinos ortogonales
    for dj, di in orthogonal:
        nj, ni = j + dj, i + di
        if 0 <= nj < ny and 0 <= ni < nx and skeleton[nj, ni]:
            neighbors.add((ni, nj))
    # 2b) Vecinos diagonales solo si ninguno de los ortogonales adyacentes ya está
    for dj, di in diagonals:
        nj, ni = j + dj, i + di
        if not (0 <= nj < ny and 0 <= ni < nx and skeleton[nj, ni]):
            continue
        ortho1 = (i, j + dj)
        ortho2 = (i + di, j)
        if ortho1 not in neighbors and ortho2 not in neighbors:
            neighbors.add((ni, nj))
    # 2c) Añade las aristas
    for nb in neighbors:
        G.add_edge(this, nb)




In [ ]:
# EXTRAEMOS LAS JUNCTIONS Y LOS TUBOS
# Una junction es un nodo con grado > 2
# Cada camino simple entre dos junctions es un tubo


# 1) Reúne todos los leaves (grado==1)
#leaves = [n for n,d in G.degree() if d == 1]

# 2) Mientras sigas teniendo leaves, quítalos todos
#while leaves:
    #G.remove_nodes_from(leaves)
    #leaves = [n for n,d in G.degree() if d == 1]

# Ahora G_pruned es tu skeleton sin dead-ends
# A partir de él:


joints = [n for n,d in G.degree() if d>2]
print(f"Número total de junctions (grado>2): {len(joints)}")

tubes = []
visited = set()
for u in joints:
    for v in G.neighbors(u):
        if (u,v) in visited or (v,u) in visited: continue
        path = [u,v]; visited.add((u,v))
        prev, curr = u, v
        # seguimos hasta la siguiente junction
        while G.degree(curr)==2:
            w = [w for w in G.neighbors(curr) if w!=prev][0]
            prev, curr = curr, w
            path.append(curr)
            visited.add((prev,curr))
        tubes.append(path)
        

In [ ]:
# Visualizamos el sistema de tubos y joints para asegurarnos de que esta todo en orden 

pos = {node:(xmin + i*dx, ymin + j*dy) for (j,i), node in zip(coords, G.nodes()) }

# Dibujamos la mascara de fondo:
fig, ax = plt.subplots(figsize=(12,8), dpi=1000)
ax.imshow(mask, origin='lower', extent=(xmin, xmax, ymin, ymax), cmap='gray', alpha=0.5)

# Dibujamos los tubos:
for tube in tubes:
    xs = [pos[n][0] for n in tube]
    ys = [pos[n][1] for n in tube]
    ax.plot(xs, ys, color='blue', linewidth=0.5)

# Dibujamos las junctions
jx = [pos[j][0] for j in joints]
jy = [pos[j][1] for j in joints]
ax.scatter(jx, jy, color='red', s=3, label='Junctions')

# Ajustes finales
ax.set_aspect('equal')
ax.set_xlabel('x(m)')
ax.set_ylabel('y(m)')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# CONSTRUIMOS EL HISTOGRAMA DE SEMIAPERTURAS BUENO (SOLO MIDPOINTS)

from scipy.stats import rayleigh, norm  

half_widths_mid = []
for tube in tubes:
    mid = len(tube)//2    # indice central
    node_mid = tube[mid]
    w_mid = G.nodes[node_mid]['width']
    half_widths_mid.append(w_mid)

half_widths_mid = np.array(half_widths_mid)
# para tenerlo todo en mm:
half_widths_mid_mm = half_widths_mid*1e3

# Ajuste Rayleigh: 
loc_r, sigma_r = rayleigh.fit(half_widths_mid_mm, floc=0)

# Ajuste normal:
mu_n, sigma_n = norm.fit(half_widths_mid_mm)

# Dominio para las curvas:
r = np.linspace(0, half_widths_mid_mm.max(), 200)

# Ploteamos el histograma junto a los fits
fig, ax = plt.subplots(figsize=(6,4), dpi=400)
ax.hist(half_widths_mid_mm, bins=30, density=True, alpha=0.6, color='C0', edgecolor='black', label='Simulation')
ax.plot(r, rayleigh.pdf(r, loc=loc_r, scale=sigma_r), 'r-', lw=2, label=f'Rayleigh fit\nσ={sigma_r:.2f} mm')
ax.plot(r, norm.pdf(r, loc=mu_n, scale=sigma_n), 'b--', lw=2, label=f'Normal fit\nμ={mu_n:.2f} mm\nσ={sigma_n:.2f} mm')

ax.set_xlabel("Half-width (mm)")
ax.set_ylabel("Probability density")
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

# Lo ploteamos tambien en escala logaritmica:
bins_log = np.logspace(np.log10(half_widths_mid_mm.min()), np.log10(half_widths_mid_mm.max()), 30)

plt.figure(figsize=(6,4), dpi=400)
plt.hist(half_widths_mid_mm, bins=bins_log, color='C1', edgecolor='black' )
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Half-width (mm)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show



In [ ]:

# ### Celda: Cálculo de flujo con midpoint “real” mediante intersección

from scipy.spatial import cKDTree
from shapely.geometry import LineString, Point
import numpy as np
import pyvista as pv

# 0) Construye KD-Tree de los pixels del skeleton podado con su width
pixel_nodes = np.array(list(G.nodes()))   # array de (i,j)
pixel_xy    = np.array([
    (xmin + i*dx, ymin + j*dy)
    for i,j in pixel_nodes
])
pixel_width = np.array([
    G.nodes[(i,j)]['width']
    for i,j in pixel_nodes
])
pix_tree = cKDTree(pixel_xy)


In [ ]:


# 1) Preparamos KDTree con los centroides de obstáculos
obs_centers = np.array([[poly.centroid.x, poly.centroid.y] for poly in obs_polys])
obs_tree    = cKDTree(obs_centers)

all_pts  = []
offsets  = [0]
tangents = []
half_ws  = []
normals = []
real_midpoints = []
L_list = []


In [ ]:

# 2) Recorremos cada tubo para generar los puntos de muestreo
for tube in tubes:
    # 2a) Índice aproximado y semiancho
    mid = len(tube) // 2
    i, j = tube[mid]
    x_idx, y_idx = pos[(i, j)]

    # 2c) Centros de obstáculos más cercanos
    _, idxs = obs_tree.query([x_idx, y_idx], k=2)
    c1, c2 = obs_centers[idxs[0]], obs_centers[idxs[1]]

    # 2d) Intersección entre el eje del tubo y la recta centros
    tube_line = LineString([pos[n] for n in tube])
    cut_line  = LineString([c1, c2])
    inter = tube_line.intersection(cut_line)
    if inter.is_empty:
        x0, y0 = x_idx, y_idx
    else:
        if isinstance(inter, Point):
            x0, y0 = inter.x, inter.y
        #else:
            # inter es un MultiPoint: extraemos sus sub-puntos con .geoms
            #pts = list(inter.geoms)
            # calculamos distancias al midpoint índice
            #dists = [Point(x_idx, y_idx).distance(pt) for pt in pts]
            # elegimos el más cercano
            #closest = pts[np.argmin(dists)]
            #x0, y0 = closest.x, closest.y

    real_midpoints.append((x0, y0))

    # 2) Encuentra la semianchura real más cercana
    dist_px, idx_px = pix_tree.query([x0, y0])
    w_real = pixel_width[idx_px]
    half_ws.append(w_real)

    # 2e) Nuevo vector normal “real”
    n_hat = c2 - c1
    n_hat /= np.linalg.norm(n_hat)

    t_hat = np.array([-n_hat[1],  n_hat[0]])

    # 3) Usa w_real para definir D y generar la línea de corte
    D = 2 * w_real
    line_ext = LineString([
        (x0 - n_hat[0]*D, y0 - n_hat[1]*D),
        (x0 + n_hat[0]*D, y0 + n_hat[1]*D),
    ])


    # 4) Intersecta con free_region y mide L
    seg = line_ext.intersection(free_region)
    if seg.is_empty:
        L = 2*w_real
        seg = LineString([
            (x0 - n_hat[0]*w_real, y0 - n_hat[1]*w_real),
            (x0 + n_hat[0]*w_real, y0 + n_hat[1]*w_real)
        ])
    elif seg.geom_type == "MultiLineString":
        seg = max(seg.geoms, key=lambda s: s.length)
    L = seg.length
    L_list.append(L)

    # 5) Muestreo equiespaciado según L
    Nsamples = max(5, int(np.ceil(L / dx)))
    ds = L / (Nsamples - 1)
    ss = [i*ds for i in range(Nsamples)]
    pts = [(seg.interpolate(s).x,
            seg.interpolate(s).y,
            z_mid) for s in ss]


    
    all_pts.extend(pts)
    offsets.append(len(all_pts))
    normals.append(n_hat)
    tangents.append(t_hat)

all_pts = np.array(all_pts)


In [ ]:

# 3) Sampleo VTK de velocidades
cloud_all = pv.PolyData(all_pts)
sampled   = vol_mesh.sample(cloud_all)
Uall      = sampled['U'][:, :2]

Uall = np.nan_to_num(Uall, nan=0.0, posinf=0.0, neginf=0.0)



In [ ]:

# 4) Cálculo de caudales por tubo usando las longitudes reales almacenadas en L_list
h = 1e-3
flow_rates = []
for k in range(len(tubes)):
    start, end = offsets[k], offsets[k+1]
    Upts = Uall[start:end]    
    t_hat = tangents[k]
    # Usamos la longitud real L_list[k] para el paso ds
    # y garantizamos que ds*(len(Upts)-1) == L_list[k]
    L_real = L_list[k]
    ds = L_real / (len(Upts) - 1) if len(Upts) > 1 else 0.0

    # Proyección de U sobre la tangente
    u_par = Upts.dot(t_hat)
    Qt = np.abs(np.sum(u_par * ds)) * h
    flow_rates.append(Qt)

print(f"Computed flow_rates for {len(flow_rates)} tubes")

In [ ]:
data = np.array(flow_rates)

fig, ax = plt.subplots(figsize=(8, 6), dpi=200)

# Definimos bins lineales para el histograma
bins = np.linspace(data.min(), data.max(), 50)

# Dibujamos el histograma
ax.hist(data, bins=bins, alpha=0.75, edgecolor='black')

# Escala logarítmica solo en el eje Y
ax.set_yscale('log')

# Etiquetas y título
ax.set_xlabel("Flow rate $Q$ (m³/s)", fontsize=12)
ax.set_ylabel("Number of tubes", fontsize=12)
ax.set_title("Histogram of flow rates", fontsize=14, pad=12)

# Añadimos rejilla horizontal para facilitar la lectura
ax.grid(which='major', axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# CALCULAMOS LA DISTRIBUCION DE CAIDAS DE PRESION

# Preparamos la lista de puntos: inicio y fin de cada tubo
end_pts = []
for tube in tubes:
    # Nodo inicial
    i0, j0 = tube[0]
    x0, y0 = xmin + i0*dx, ymin + j0*dy
    end_pts.append((x0, y0, z_mid))
    # Nodo final
    i1, j1 = tube[-1]
    x1, y1 = xmin + i1*dx, ymin + j1*dy
    end_pts.append((x1, y1, z_mid))

end_pts = np.array(end_pts)

# Muestreamos de golpe
cloud_end = pv.PolyData(end_pts)
sampled_p = vol_mesh.sample(cloud_end)
p_vals    = sampled_p['p']  # array de longitud 2*N_tubes

# Calculamos la caída de presión por tubo
N = len(tubes)
dp = np.empty(N)
for k in range(N):
    p_start = p_vals[2*k]
    p_end   = p_vals[2*k + 1]
    dp[k]   = abs(p_start - p_end)

# Ahora, calculamos la caida de presion teorica (asumiendo Poisueuille)
# Para ello, en primer lugar, calculamos la longitud de cada tubo:
lengths = []
for tube in tubes:
    L = 0.0
    for k in range(len(tube)-1):
        i1, j1 = tube[k]
        i2, j2 = tube[k+1]
        p1 = np.array([xmin + i1*dx, ymin + j1*dx])
        p2 = np.array([xmin + i2*dx, ymin + j2*dx])
        L += np.linalg.norm(p2 - p1)
    lengths.append(L)
lengths = np.array(lengths)

# Definimos la viscosidad (agua a 20°C)
mu = 1e-3  # Pa·s

# Caída de presión teórica (MC Poiseuille en cabina)
Q = np.array(flow_rates)
w = np.array(half_ws)
dp_pois = 3 * mu * lengths * Q / (2 * w**3)

# Histograma (comparativo) de caídas de presión
min_nonzero = min(dp[dp>0].min(), dp_pois[dp_pois>0].min())
max_value   = max(dp.max(), dp_pois.max())
bins = np.logspace(np.log10(min_nonzero), np.log10(max_value), 30)

fig, ax = plt.subplots(figsize=(8,6), dpi=200)
ax.hist(dp, bins=bins, alpha=0.6, edgecolor='black', label='Real ΔP (OpenFOAM)')
ax.hist(dp_pois, bins=bins, histtype='step', linestyle='--', linewidth=2, color='red', label='Estimated ΔP (Poiseuille)')

# 3) Log–log
ax.set_xscale('log')
ax.set_yscale('log')

# 4) Etiquetas y estética
ax.set_xlabel("Pressure drop ΔP (Pa)", fontsize=12)
ax.set_ylabel("Number of tubes", fontsize=12)
ax.legend()
ax.grid(which='both', linestyle='--', linewidth=0.5, alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
# CALCULAMOS LA POROSIDAD:
phi = mask.mean()  
print(f"Porosidad φ = {phi:.4f}")

In [ ]:
# REPRESENTAMOS LOS COLOREADOS SEGUN SU CAUDAL

from matplotlib.collections import LineCollection
from matplotlib.colors import LogNorm, LinearSegmentedColormap


segments   = [np.array([pos[n] for n in tube]) for tube in tubes]
flow_array = np.array(flow_rates)

# Colormap y escala log
cmap = LinearSegmentedColormap.from_list("BlueRed", ["blue","red"])
norm = LogNorm(vmin=flow_array[flow_array>0].min(), vmax=flow_array.max())

lc = LineCollection(segments, cmap=cmap, norm=norm, linewidth=1.5)
lc.set_array(flow_array)

# A pintar
fig, ax = plt.subplots(figsize=(12,8), dpi=400)

ax.imshow(mask, origin='lower', extent=(xmin, xmax, ymin, ymax), cmap='Greys_r', alpha=0.4)
ax.add_collection(lc)


ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")

cbar = fig.colorbar(lc, ax=ax, shrink=0.5, pad=0.02)
cbar.set_label("Flow rate $Q$ (m³/s)")

plt.tight_layout()
plt.show()

In [ ]:
# CALCULAMOS LA DISTRIBUCION DE BEAD SIZES:

radii_obs = np.array([ np.sqrt(poly.area / np.pi) for poly in obs_polys])

# los pasamos a mm:
radii_mm = radii_obs * 1e3
print(f"Nº de obstáculos: {len(radii_mm)}")
print(f"Radio mínimo: {radii_mm.min():.3f} mm, máximo: {radii_mm.max():.3f} mm")



# 4) Histograma de radios de obstáculos
plt.figure(figsize=(6,4), dpi=150)
plt.hist(radii_mm, bins=40, edgecolor='black', alpha=0.8)
plt.xlabel("Bead radius (mm)", fontsize=12)
plt.ylabel("Number of beads", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.yscale('log')
plt.tight_layout()
plt.show()

In [ ]:
# Calculamos la correlacion de Pearson en el tamano de los beads

from scipy.spatial.distance import pdist, squareform
# Los radios ya los tenemos (en radii_mm), pero necesitamos los centros de cada obstaculo:
centers = np.array([[poly.centroid.x, poly.centroid.y] for poly in obs_polys])

r_mean = radii_mm.mean()
print(r_mean)
r_var = radii_mm.var()
delta_r = radii_mm - r_mean

# Distancias y productos de fluctuaciones dos a dos:
dists = pdist(centers) # distancias en metros
outer = np.outer(delta_r, delta_r)
i_upper = np.triu_indices(len(radii_mm), k=1)
prods = outer[i_upper]

# pasamos las distancias a mm:
dists_mm = dists*1e3

nbins = 500
max_d = 60.0 #mm
min_d = dists_mm.min()
bins = np.linspace(min_d, max_d, nbins+1)
bin_centers = 0.5*(bins[:-1] + bins[1:])


# computamos C(d)
sum_prod, _ = np.histogram(dists_mm, bins=bins, weights=prods)
counts, _ = np.histogram(dists_mm, bins=bins)
C_d = np.zeros_like(bin_centers)
mascara = counts>0
C_d[mascara] = sum_prod[mascara]/(counts[mascara]*r_var)

# A pintar!
plt.figure(figsize=(6,4), dpi=400)
plt.plot(bin_centers, C_d, '-', linewidth=1, color='C0')
plt.xlim(min_d, 20)
plt.xlabel("Distance $d$ (mm)", fontsize=12)
plt.ylabel("Spatial correlation $C(d)$", fontsize=12)
plt.grid(ls='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# — Celda X (adaptada): Correlación de caudales en junctions de grado 3 usando real_midpoints —
import numpy as np
import matplotlib.pyplot as plt

dz = 1e-3  # espesor en z fijo

# 1) Datos ya definidos en celdas anteriores:
#    flow_rates, tubes, G_pruned, real_midpoints (lista de (x0,y0) para cada tubo)
#    tangents          (lista de t_hat unitarios ya orientados TODO el bucle anterior)
#    offsets, all_pts, Uall

flows = np.array(flow_rates)

# 2) Índice nodo→tubos (sin cambios)
tube_index = {}
for idx, tube in enumerate(tubes):
    for node in tube:
        tube_index.setdefault(node, []).append(idx)

# 3) Detectar las junctions de grado 3 en el grafo podado
junc3 = [n for n, d in G.degree() if d == 3]

# 3.1) Reorientación de tangentes según proyección media de Uall (igual que antes)
n_tubes   = len(tubes)
proj_mean = np.empty(n_tubes)
for i in range(n_tubes):
    start, end = offsets[i], offsets[i+1]
    v_sec      = Uall[start:end]         # velocidades XY en la sección
    proj_mean[i] = v_sec.dot(tangents[i]).mean()
oriented_tangents = tangents * np.sign(proj_mean)[:, None]

# 4) Inicializar contadores
sum_prods      = 0.0
count_prods    = 0
type1_count    = 0
type2_count    = 0
sum_feed_prods = 0.0
n_feed_cases   = 0

# 5) Clasificación “hacia/desde la junction” usando real_midpoints
for j in junc3:
    idxs = tube_index.get(j, [])
    if len(idxs) != 3:
        continue

    qs    = flows[idxs]
    i_max = idxs[np.argmax(qs)]

    
    

    # --- Aquí usamos real_midpoints en lugar de pos[mid_node] ---
    x0, y0   = real_midpoints[i_max]       # sección del tubo i_max
    xj, yj   = pos[j]                      # posición de la junction
    v2j      = np.array((xj - x0, yj - y0)) # vector sección→junction

    dp = oriented_tangents[i_max].dot(v2j)

    others = [i for i in idxs if i != i_max]
    if len(others) < 2:
        # caso raro: no hay dos tubos además del máximo, lo saltamos
        continue
    if dp > 0:
        # caso 2: el tubo i_max es feeder → dos receptores
        for o in others:
            sum_prods   += flows[i_max] * flows[o]
            count_prods += 1
        type2_count += 1
    else:
        # caso 1: el tubo i_max es receptor ← dos feeders
        q1, q2 = flows[others[0]], flows[others[1]]
        sum_prods      += q1 * q2
        count_prods    += 1
        type1_count    += 1
        sum_feed_prods += q1 * q2
        n_feed_cases   += 1

# (Opcional) mostrar figura de contexto, si quieres:
plt.tight_layout()
plt.show()

# 6) Pearson global
mean_q   = flows.mean()
E_qiqj   = sum_prods / count_prods
cov_qiqj = E_qiqj - mean_q**2
var_q    = flows.var()
pearson  = cov_qiqj / var_q if var_q != 0 else np.nan

print(f"Junctions tipo1 (2→1): {type1_count}")
print(f"Junctions tipo2 (1→2): {type2_count}")
print(f"E[q_i·q_j] global       = {E_qiqj:.3e}")
print(f"Pearson global          = {pearson:.3e}")

# 7) Pearson solo feeders (caso1)
if n_feed_cases > 0:
    E_feed_prod  = sum_feed_prods / n_feed_cases
    cov_feed     = E_feed_prod - mean_q**2
    pearson_feed = cov_feed / var_q if var_q != 0 else np.nan
    print(f"Casos tipo1             = {n_feed_cases}")
    print(f"E[q1*q2] feeders        = {E_feed_prod:.3e}")
    print(f"Pearson feeders (caso1) = {pearson_feed:.3e}")
else:
    print("No hay casos tipo1 para Pearson feeders.")

# 8) Diagnósticos adicionales
print("0.75 Var + (9/16) mean² =", 0.75*var_q + 9/16*mean_q**2)
print("mean_q =", mean_q)
print("mean_q²/var_q =", mean_q**2/var_q)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# %% [markdown]
# ### Celda X+1 (mejorada): secciones con vectores U reescalados y flechas amarillas en junction

# %%
# Asumimos que ya tienes definidas:
# tubes, offsets, all_pts, Uall, real_midpoints, oriented_tangents, tube_index, junc3, pos, mask,
# xmin, xmax, ymin, ymax

# 1) Escalado de vectores U para visibilidad
# hallamos la magnitud máxima en Uall
vel_mags = np.linalg.norm(Uall, axis=1)
max_vel = vel_mags.max() if vel_mags.max()>0 else 1.0
# definimos factor de escala para que el vector más grande tenga longitud arrow_len
arrow_len = (xmax - xmin) * 0.002
vel_scale = 5*arrow_len / max_vel
U_scaled = Uall * vel_scale  # ahora visibles

# 2) Clasificación de junctions
junc1, junc2 = [], []
for j in junc3:
    idxs = tube_index.get(j, [])
    if len(idxs) != 3:
        continue
    qs = np.array([flow_rates[i] for i in idxs])
    i_max = idxs[np.argmax(qs)]
    x0, y0 = real_midpoints[i_max]
    xj, yj = pos[j]
    dp = oriented_tangents[i_max].dot((xj-x0, yj-y0))
    if dp > 0:
        junc2.append(j)
    else:
        junc1.append(j)

# 3) Datos para flechas verdes (tangentes)
Xg, Yg, Ug, Vg = [], [], [], []
for k in range(len(tubes)):
    x0, y0 = real_midpoints[k]
    tx, ty = oriented_tangents[k]
    Xg.append(x0); Yg.append(y0)
    Ug.append(tx * arrow_len); Vg.append(ty * arrow_len)

# 4) Datos para flechas amarillas (junction center)
Xy, Yy, Uy, Vy = [], [], [], []
for j in junc3:
    idxs = tube_index[j]
    qs = np.array([flow_rates[i] for i in idxs])
    i_max = idxs[np.argmax(qs)]
    xj, yj = pos[j]  # junction center
    tx, ty = oriented_tangents[i_max]
    Xy.append(xj); Yy.append(yj)
    Uy.append(tx * arrow_len); Vy.append(ty * arrow_len)

# 5) Gráfico
fig, ax = plt.subplots(figsize=(12, 8), dpi=1200)
ax.imshow(mask, origin='lower',
          extent=(xmin, xmax, ymin, ymax),
          cmap='gray', alpha=0.4)

# 6) Secciones y vectores U reescalados
for k in range(len(tubes)):
    start, end = offsets[k], offsets[k+1]
    pts_k = all_pts[start:end]
    uscaled = U_scaled[start:end]
    xs, ys = pts_k[:,0], pts_k[:,1]
    # sección
    ax.plot(xs, ys, color='black', linewidth=0.5, alpha=0.6, zorder=1)
    # vectores de velocidad
    ax.quiver(xs, ys,
              uscaled[:,0], uscaled[:,1],
              angles='xy', scale_units='xy', scale=1,
              width=0.0002, color='cyan', alpha=0.8, zorder=2)

# 7) Flechas verdes (tangentes)
ax.quiver(
    Xg, Yg, Ug, Vg,
    angles='xy', scale_units='xy', scale=1,
    width=0.0004, headwidth=1, headlength=1.2, headaxislength=1,
    color='green', alpha=0.8, zorder=3
)

# 8) Flechas amarillas (tube máximo en junction)
ax.quiver(
    Xy, Yy, Uy, Vy,
    angles='xy', scale_units='xy', scale=1,
    width=0.0004, headwidth=1, headlength=1.2, headaxislength=1,
    color='yellow', alpha=0.9, zorder=4
)

# 9) Etiquetas pequeñas para junciones
for j in junc1:
    xj, yj = pos[j]
    ax.text(xj, yj, "1", color='red', fontsize=2,
            fontweight='bold', ha='center', va='center', zorder=5)
for j in junc2:
    xj, yj = pos[j]
    ax.text(xj, yj, "2", color='blue', fontsize=2,
            fontweight='bold', ha='center', va='center', zorder=5)

ax.set_aspect('equal')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Secciones, vectores U reescalados y tangentes (verde/amarillo)')
plt.tight_layout()
plt.show()


In [ ]:
# Selecciona el índice del tubo a inspeccionar
k = 6000  # Cambia este valor para otros tubos

# Extrae puntos y velocidades para el tubo k
start, end = offsets[k], offsets[k+1]
pts_k = all_pts[start:end]    # (N_k, 3) coordenadas (x, y, z_mid)
U_k   = Uall[start:end]       # (N_k, 2) componentes (U_x, U_y)

# Crea figura para depuración
fig, ax = plt.subplots(figsize=(6, 6), dpi=300)
# Dibuja la sección del tubo
ax.plot(pts_k[:, 0], pts_k[:, 1], '-k', linewidth=1, label=f'Tubo {k} sección')

# Dibuja vectores de velocidad en cada punto
ax.quiver(
    pts_k[:, 0], pts_k[:, 1],
    U_k[:, 0], U_k[:, 1],
    angles='xy', scale_units='xy', scale=1,
    width=0.002, color='red', alpha=0.8, label='Vectores U'
)

ax.set_aspect('auto')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title(f'Sección y velocidad en tubo {k}')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

# 1) Extrae y triangulariza la superficie
surf = vol_mesh.extract_surface()
surf_tri = surf.triangulate()  # ahora solo triángulos

# 2) Puntos XY
pts = surf_tri.points[:, :2]

# 3) Caras (cada tri es [3, i0, i1, i2])
faces = surf_tri.faces.reshape(-1, 4)[:, 1:4]

# 4) Dibujo 2D
fig, ax = plt.subplots(figsize=(6,6), dpi=300)
for tri in faces:
    poly = pts[tri]
    # cerrar el polígono
    xs = np.append(poly[:,0], poly[0,0])
    ys = np.append(poly[:,1], poly[0,1])
    ax.plot(xs, ys, color='black', linewidth=0.3)

ax.set_aspect('equal')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Proyección XY de la superficie de la malla (triangularizada)')
plt.tight_layout()
plt.show()


In [ ]:
# — Celda 1: Gradiente del campo U en celdas —
import vtk
from vtk.util.numpy_support import vtk_to_numpy

# Aplica vtkGradientFilter sobre el UnstructuredGrid vol_mesh
grad_filter = vtk.vtkGradientFilter()
grad_filter.SetInputData(vol_mesh)
# Asocia 'U' a cada celda
grad_filter.SetInputScalars(vtk.vtkDataSet.FIELD_ASSOCIATION_CELLS, 'U')
grad_filter.SetResultArrayName('gradU')
grad_filter.Update()

# Envuelve el resultado en PyVista
grad_mesh = pv.wrap(grad_filter.GetOutput())

# Extraemos el tensor gradiente: cada fila es [∂Ux/∂x, ∂Ux/∂y, …, ∂Uz/∂z]
gradU = grad_mesh.cell_data['gradU']  


In [ ]:
# — Celda 2 (corregida): Centros de celda, velocidades y volúmenes —
# Centros de cada celda
cell_centers = grad_mesh.cell_centers().points  # (n_cells, 3)

# Velocidad en cada celda (asumiendo que tienes U en cell_data; 
# si no, muestrea point_data sobre cell_data_centers)
if 'U' in grad_mesh.cell_data:
    U_cell = grad_mesh.cell_data['U']
else:
    cloud = pv.PolyData(cell_centers)
    samp  = vol_mesh.sample(cloud)
    U_cell = samp.point_data['U']

# Magnitud de la velocidad en el plano XY
vel_mag = np.linalg.norm(U_cell[:, :2], axis=1)

# Volumen de cada celda (en m³)
# usamos compute_cell_sizes() en lugar de compute_cell_size()
sizes_mesh = grad_mesh.compute_cell_sizes()  
vols = sizes_mesh.cell_data['Volume']


In [ ]:
# — Celda 3: Componentes del gradiente en (w,z) —
# Desempaquetamos los 9 componentes de gradU
dxx, dxy, dxz, dyx, dyy, dyz, dzx, dzy, dzz = gradU.T

# Arrays de salida
n_cells = len(vel_mag)
delww = np.zeros(n_cells)
delwz = np.zeros(n_cells)
delzw = np.zeros(n_cells)
delzz = np.zeros(n_cells)

for idx in range(n_cells):
    ux, uy = U_cell[idx,0], U_cell[idx,1]
    vm2 = ux*ux + uy*uy
    if vm2 == 0.0:
        continue

    cos2   = ux*ux/vm2
    sin2   = 1.0 - cos2
    sincos = ux*uy/vm2
    alpha  = dxy[idx] + dyx[idx]
    beta   = dyy[idx] - dxx[idx]

    # formúlulas del C++
    delww[idx] = dxx[idx]*cos2 + dyy[idx]*sin2 + alpha*sincos
    delwz[idx] = dxy[idx]*cos2 - dyx[idx]*sin2 + beta*sincos
    delzw[idx] = dyx[idx]*cos2 - dxy[idx]*sin2 + beta*sincos
    delzz[idx] = dyy[idx]*cos2 + dxx[idx]*sin2 - alpha*sincos


In [ ]:
# — Celda 4: Función de histograma ponderado logarítmico y guardado —
def weighted_log_hist(values, weights, N, vmin, vmax):
    bins = np.logspace(np.log10(vmin), np.log10(vmax), N+1)
    hist, edges = np.histogram(values, bins=bins, weights=weights)
    centers = np.sqrt(edges[:-1] * edges[1:])
    return hist, centers

N = 40

# 1) Histograma de |Ux,y| (velocidad)
minU, maxU = 1e-7, vel_mag.max()
hU, cU = weighted_log_hist(vel_mag, vols, N, minU, maxU)
np.savetxt('histograma_U.dat', np.column_stack((cU, hU)),
           header='velocity_mag[m/s]\thistogram')

# 2) Histograma de shearXY = |∂Ux/∂y|
shearXY = np.abs(dxy)
minS, maxS = 1e-4, shearXY.max()
hS, cS = weighted_log_hist(shearXY, vols, N, minS, maxS)
np.savetxt('histograma_shear_XY.dat', np.column_stack((cS, hS)),
           header='|dUx/dy|[1/s]\thistogram')

# 3) Análogos para delww, delwz, delzw, delzz…
for arr, name in [(delww,'WW'), (delwz,'WZ'), (delzw,'ZW'), (delzz,'ZZ')]:
    mn, mx = 1e-4, np.abs(arr).max()
    h, c = weighted_log_hist(np.abs(arr), vols, N, mn, mx)
    np.savetxt(f'histograma_shear_{name}.dat', np.column_stack((c, h)),
               header=f'|d{name}|[1/s]\thistogram')


In [ ]:
# — Celda 5: 2D log-histograma y shear dado velocidad —
# Construimos bins log en ambas direcciones
xbins = np.logspace(np.log10(minU), np.log10(maxU), N+1)
ybins = np.logspace(np.log10(minS), np.log10(maxS), N+1)

# 2D histograma ponderado
hist2d, xedges, yedges = np.histogram2d(
    vel_mag, np.abs(delwz), bins=[xbins, ybins], weights=np.repeat(vols, 1)
)

# Centroides de bin
xcent = np.sqrt(xedges[:-1]*xedges[1:])
ycent = np.sqrt(yedges[:-1]*yedges[1:])

# shear promedio y error en cada bin de velocidad
shear_given_v = np.zeros(N)
sh_error_v   = np.zeros(N)
for i in range(N):
    mask = (vel_mag>=xedges[i]) & (vel_mag<xedges[i+1])
    w = vols[mask]
    s = np.abs(delwz[mask])
    if w.sum()>0:
        # suma de pesos
        wsum = w.sum()
        # media ponderada
        mean_s = np.sum(s * w) / wsum
        # varianza ponderada
        var_w = np.sum(w * (s - mean_s)**2) / wsum
        # cálculo del error estándar ponderado:
        # N_eff = (sum w)^2 / sum(w^2)
        # err = sqrt(var_w / N_eff) = sqrt(var_w * sum(w^2) / sum(w)^2)
        err_s = np.sqrt(var_w * np.sum(w**2) / wsum**2)

        shear_given_v[i] = mean_s
        sh_error_v[i]   = err_s
# Guardado
np.savetxt('shear_given_v.dat', np.column_stack((xcent, shear_given_v, sh_error_v)),
           header='velocity_bin_center[m/s]\tshear_mean[1/s]\tshear_std[1/s]')
np.savetxt('conditioned_2D_histo.dat', np.column_stack(
    [hist2d.flatten(), 
     np.repeat(xcent, N), 
     np.tile(ycent, N)]
), header='hist_val\tvel_center\tshear_center')


In [ ]:
# — Celda 6: Promedios ponderados por volumen —
avgVel   = np.sum(vel_mag * vols) / np.sum(vols)
avgShear = np.sum(np.abs(delwz) * vols) / np.sum(vols)

with open('averages.dat', 'w') as f:
    f.write('# Promedio ponderado por volumen\n')
    f.write(f'avgVelocity  {avgVel:.6e}\n')
    f.write(f'avgShearWZ   {avgShear:.6e}\n')
print("Promedios guardados en 'averages.dat'")


In [ ]:
# — Celda 7 (actualizada): Histogramas con valores mínimos fijos —
import matplotlib.pyplot as plt
import numpy as np

# Parámetros
nbins = 50
vmin = 1e-6
smin = 1e-3

# Filtramos para evitar valores por debajo del umbral
vel_filt   = vel_mag[vel_mag >= vmin]
shear_wz_f = np.abs(delwz)
shear_filt = shear_wz_f[shear_wz_f >= smin]

# Máximos para los ejes
vmax = vel_filt.max()
smax = shear_filt.max()

# 1) Histograma de vel_mag en escala lineal
fig, ax = plt.subplots(figsize=(6,4))
ax.hist(vel_filt, bins=np.linspace(vmin, vmax, nbins), 
        density=True, edgecolor='black')
ax.set_xlabel('Velocidad $|\\mathbf{U}|$ (m/s)')
ax.set_ylabel('Número de celdas')
ax.set_title('Histograma velocidad (lin–lin)')
plt.tight_layout()
plt.show()

# 2) Histograma de vel_mag en escala log–log
fig, ax = plt.subplots(figsize=(6,4))
ax.hist(vel_filt, bins=np.logspace(np.log10(vmin), np.log10(vmax), nbins),
        density=True, edgecolor='black')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Velocidad $|\\mathbf{U}|$ (m/s)')
ax.set_ylabel('Número de celdas')
ax.set_title('Histograma velocidad (log–log)')
plt.tight_layout()
plt.show()

# 3) Histograma de shear WZ en escala lineal
fig, ax = plt.subplots(figsize=(6,4))
ax.hist(shear_filt, bins=np.linspace(smin, smax, nbins),
        density=True, edgecolor='black')
ax.set_xlabel('|$∂_{WZ}$| (1/s)')
ax.set_ylabel('Número de celdas')
ax.set_title('Histograma shear WZ (lin–lin)')
plt.tight_layout()
plt.show()

# 4) Histograma de shear WZ en escala log–log
fig, ax = plt.subplots(figsize=(6,4))
ax.hist(shear_filt, bins=np.logspace(np.log10(smin), np.log10(smax), nbins),
        density=True, edgecolor='black')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('|$∂_{WZ}$| (1/s)')
ax.set_ylabel('Número de celdas')
ax.set_title('Histograma shear WZ (log–log)')
plt.tight_layout()
plt.show()


In [ ]:
# — Celda 8: Promedio de shear WZ dado velocidad (log–log) —

# (Asumimos que ya tienes xcent, shear_given_v y sh_error_v de la Celda 5)

fig, ax = plt.subplots(figsize=(6,4))
ax.errorbar(xcent, shear_given_v, yerr=sh_error_v, fmt='o', markersize=4,
            elinewidth=1, capsize=2, label='⟨shear WZ⟩ por bin de velocidad')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Velocidad bin (m/s)')
ax.set_ylabel('Promedio |$∂_wz$| (1/s)')
ax.set_title('Shear WZ promedio vs velocidad (log–log)')
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
#Ajuste de caudales y comparación de P(v) y S(v) teórico/experimental —

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon
from scipy.special import k0

# 0) Parámetros geométricos
h = 0.5e-3                   # espesor h = 1 mm
r = h/np.sqrt(12)          # r = h/√12
a = np.mean(half_ws)       # semianchura promedio (m)

# 1) Ajuste exponencial de la distribución de caudales → Qc
flow = np.array(flow_rates)
_, Qc = expon.fit(flow, floc=0)
print(f"Qc (escala parámetro exponencial) = {Qc:.3e} m³/s")

# 2) Cálculo de w_c
wc = 3*Qc*a / (h**3 * (1 - (r/a)*np.tanh(a/r)))
print(f"w_c = {wc:.3e} m/s")

# 3) Histograma de caudales + ajuste exponencial
Qv = np.linspace(flow.min(), flow.max(), 200)
pdf_exp = (1/Qc) * np.exp(-Qv/Qc)

plt.figure(figsize=(6,4))
plt.hist(flow, bins=30, density=True, alpha=0.6,
         edgecolor='black', label='Caudales (histograma)')
plt.plot(Qv, pdf_exp, 'r-', lw=2, label='Ajuste exp')
plt.xlabel('Caudal $Q$ (m³/s)')
plt.ylabel('Densidad')
plt.title('Distribución de caudales y ajuste exponencial')
plt.legend()
plt.tight_layout()
plt.show()

# 4) Preparación para P(v) y S(v)
v = np.array(vel_mag)
vmin = 1e-7
mask = v >= vmin
v_filt = v[mask]
v_th = np.linspace(vmin, v.max(), 300)

# 5) Constantes intermedias
ar      = a/r
cosh_ar = np.cosh(ar)
sinh_ar = np.sinh(ar)
coth_ar = cosh_ar/sinh_ar
factor  = cosh_ar / (sinh_ar**2)   # = cosh(ar)/sinh²(ar)
alpha2  = (a/r)**2

# 6) P_teórico(v) según tu foto
P_pref = ( (a/r) * coth_ar ) / (2*wc)
P_th   = P_pref * np.exp( -0.5 * alpha2 * coth_ar**2 * (v_th/wc) ) * k0( 0.5 * factor * alpha2 * (v_th/wc) )

# 7) S_teórico(v) según tu foto
#    S(v) = [2 w_c/a * (r/a) tanh(a/r)] 
#           * exp{ -½ [cosh(ar)/sinh²(ar)] (a/r)² v/wc }
#           * K0[+½ [cosh(ar)/sinh²(ar)] (a/r)² v/wc]
S_pref = (2*wc/a) * (r/a) * np.tanh(ar)
S_th   = S_pref * np.exp( -0.5 * factor * alpha2 * (v_th/wc) )/k0( 0.5 * factor * alpha2 * (v_th/wc) )

# 8) Gráfica de P(v)
plt.figure(figsize=(6,4))
bins_v = np.logspace(np.log10(vmin), np.log10(v.max()), 40)
plt.hist(v_filt, bins=bins_v, density=True, alpha=0.6,
         edgecolor='black', label='P_exp (histograma)')
plt.plot(v_th, P_th, 'r-', lw=2, label='P_teórico')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Velocidad $v$ (m/s)')
plt.ylabel('Densidad $P(v)$')
plt.title('Distribución de velocidades (v ≥ 1e-7 m/s)')
plt.legend()
plt.tight_layout()
plt.show()

# 9) Gráfica de S(v)
plt.figure(figsize=(6,4))
plt.errorbar(xcent, shear_given_v, yerr=sh_error_v,
             fmt='o', markersize=4, elinewidth=1, capsize=2,
             label='⟨shear WZ⟩ experimental')
plt.plot(v_th, S_th, 'r--', lw=2, label='S_teórico')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Velocidad $v$ (m/s)')
plt.ylabel('Shear $S(v)$ (1/s)')
plt.title('Shear WZ vs velocidad (log–log)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# — Celda X: Comparativa experimental vs teórico en escala lineal (lin–lin) —

import numpy as np
import matplotlib.pyplot as plt

# Reutilizamos variables de la celda anterior: 
# vel, vmin, vel_filt, v_th, P_th, xcent, shear_given_v, sh_error_v, S_th

# 1) Histograma de P(v) lin–lin
plt.figure(figsize=(6,4))
bins_lin = np.linspace(vmin, vel.max(), 40)
plt.hist(vel_filt, bins=bins_lin, density=True, alpha=0.6,
         edgecolor='black', label='P_exp (histograma)')
plt.plot(v_th, P_th, 'r-', lw=2, label='P_teórico')
plt.xlabel('Velocidad $v$ (m/s)')
plt.ylabel('Densidad $P(v)$')
plt.title('Distribución de velocidades (lin–lin)')
plt.legend()
plt.tight_layout()
plt.show()

# 2) Comparativa S(v) lin–lin
plt.figure(figsize=(6,4))
plt.errorbar(xcent, shear_given_v, yerr=sh_error_v,
             fmt='o', markersize=4, elinewidth=1, capsize=2,
             label='S_exp = ⟨shear WZ⟩')
plt.plot(v_th, S_th, 'r--', lw=2, label='S_teórico')
plt.xlabel('Velocidad $v$ (m/s)')
plt.ylabel('Shear $S(v)$ (1/s)')
plt.title('Shear WZ vs velocidad (lin–lin)')
plt.legend()
plt.tight_layout()
plt.show()
